# Simple Evolutionary Algorithm
Inspired by [Genetic Algorithm Cartpole](https://github.com/mymultiverse/GeneticAlgo_OpenAIGymCartPole). I've been wanting to play with OpenAI Gym for quite sometime and never could fully understand RL Agents. When I read breifly about evolutionary alrgorithms they made much more conceptual sense to me that I really dug right in. 

This is my interpretation of an Evolutionary Algorithm. I have no education in them other than mimicking the examples I found online and adding my own modifications. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.stats import rankdata
%matplotlib inline
import random
import matplotlib.pyplot as plt
import math
import os, sys

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras import regularizers

In [47]:
import gym

env = gym.make('CartPole-v0')
obs = env.reset()

In [161]:
def create_model(layers=[4,4,2,1]):
    model = Sequential()
    model.add(Dense(layers[1],input_shape=(layers[0],),activation='relu'))
    for neurons_in_layer in layers[2:-1]:
        model.add(Dense(neurons_in_layer,activation='relu'))
    model.add(Dense(layers[-1],activation='sigmoid'))
    return model

def experience_env(env,individual):
    obs = env.reset()
    award = 0
    done = False
    while done == False:
        action = individual.predict_classes(s(obs))
        #action = individual.predict_proba(s(obs))
        # action: (array([ 0.03162412,  0.15706954,  0.0446083 , -0.28510563]), 1.0, False, {})
        obs, reward, done, info = env.step(action[0][0])
        award += reward
    return award

def s(obs,num=4):
    return np.reshape(obs, [1, num])

In [135]:
experience_env(env,create_model())

19.658689189203734

In [160]:
def create_first_gen(num_individuals=10,layers=[4,6,5,2,1],output_activation='sigmoid'):
    generation = []
    for i in range(num_individuals):
        model = create_model(layers)
        model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=.1),metrics=['accuracy'])
        generation.append(model)
    return generation

The next few functions are a side project I tried after creating the evolutionary algorithm. I basically wanted to influence the model to dance or sway as it balanced the pole - so i added an extra bit of reward to the total reward to be gained if the model followed a sin or tan wave corresponding to its velocity or pole angle. 

The difficulty I found was weighing both the reward to survive in addition to the dancing. Too low and the model would fall into a local optima of always recieving full points on pole angle until it fell ~30 frames out. Too high and the model wouldn't behave any different than the originals. 

In the first few generations I found that the dancing had more influence than lter on. I'd love to play with it more but I'm off to learn more Q-learning as well. Maybe someday I'll be required to create a multivariate reward function and then I'll get to practice again. 

In [159]:
# def create_tan(fs=600,f=10):
#     x = np.arange(fs)
#     sin = np.array([np.sin(2*np.pi*f*(i/fs)) for i in x])
#     cos = np.array([np.cos(2*np.pi*f*(i/fs)) for i in x])
#     tan = np.array([np.tan(2*np.pi*f*(i/fs)) for i in x])
#     tan[tan > 100] = 0
#     tan = tan/(tan.max()*50) # best f=4
#     saw = np.array([signal.sawtooth(2*np.pi*f*(i/fs)) for i in x])
#     saw = saw/27 # best f=10
#     #return sin/40
#     return tan

# def dance_reward(observed_run):
#     df = pd.DataFrame(observed_run)
#     df.columns = ['Cart Position','Cart Velocity','Pole Angle','Pole Velocity']
    
#     for f in range(3,7):
#     #    sig = create_tan(fs=600,f=4) # tan
#         sig = create_tan(fs=600,f=f) # saw

#         limit = .01
#         sig_mask = np.abs(np.copy(sig))
#         sig_mask[sig_mask > limit] = 1
#         sig_mask[sig_mask < limit] = 0

#     #    sig = sig*sig_mask

#         target = 'Cart Velocity'
#         shift = df[target].shape[0]
#         pole_angle_rewards = np.array([])

#         start = 0
#         end = shift

#         for i in range(shift):
#             shifted_reward = np.sum(np.abs(sig[start:end]-df[target].values))
#             inv_shifted_reward = np.sum(np.abs((-1*sig[start:end])-df[target].values))
#             pole_angle_rewards = np.append(pole_angle_rewards,shifted_reward)
#             pole_angle_rewards = np.append(pole_angle_rewards,inv_shifted_reward)
#             start += 1
#             end += 1

#     return pole_angle_rewards.min()

# def experience_env(env,individual,dance_scale=100):
#     obs = env.reset()
#     award = 0
#     observed_run = []
#     done = False
#     while done == False:
#         action = individual.predict_classes(s(obs))
#         #action = individual.predict_proba(s(obs))
#         # action: (array([ 0.03162412,  0.15706954,  0.0446083 , -0.28510563]), 1.0, False, {})
#         obs, reward, done, info = env.step(action[0][0])
#         observed_run.append(obs)
#         award += reward
#     #return (dance_scale/dance_reward(observed_run))+award
#     return award

load model from below for single example

In [157]:
model = create_first_gen()[0]
#model.load_weights('1000.cartpole')
experience_env(env,model)

9.0

In [156]:
def generation_experience_env(env,generation,trys=5):
    generation_reward = []

    for individual in generation:
        individual_trys_reward = []
        for i in range(trys):
            individual_trys_reward.append(experience_env(env,individual))
        generation_reward.append(individual_trys_reward)
    return generation,np.array(generation_reward)

should go with the median over the sum, clearly some models get 'lucky' and aren't consistent

    ([<keras.models.Sequential at 0x1c14d112b0>,
      <keras.models.Sequential at 0x1c14bd85f8>,
      <keras.models.Sequential at 0x1c14d01240>,
      <keras.models.Sequential at 0x1c1498d470>,
      <keras.models.Sequential at 0x1c1534e710>],
     array([[  9.,   9.,   9.,  10.,  10.,   9.,   8.,   9.,   8.,  10.],
            [ 73.,  53.,  88.,  68.,  82.,  62., 109.,  52., 104.,  88.],
            [  8.,   9.,   8.,  10.,  10.,  11.,   9.,   9.,  13.,  10.],
            [  9.,  27.,  10.,  21.,  25.,  13.,   8.,  10.,  21.,  10.],
            [171.,  23.,  41., 200.,  22.,  14.,  38.,  51.,  75.,  31.]]))

In [137]:
gen = create_first_gen(10)
gen,gen_reward = generation_experience_env(env,gen,10)
gen,gen_reward

([<keras.models.Sequential at 0x10ba96550>,
 array([[ 20.15106472,  19.5523277 ,  19.56489165,  20.71931307,
          22.53922186,  19.1728766 ,  20.13360245,  18.97748606,
          21.893091  ,  19.0612335 ],
        [ 22.3287656 ,  19.91410488,  19.63848349,  20.18279431,
          21.4513794 ,  20.89570993,  22.05606179,  19.26677988,
          22.54112543,  19.97197681],
        [ 60.62601847, 174.78372983,  67.67988314,  44.33174908,
          65.03546833,  44.895755  ,  40.59404468,  62.86942585,
          56.04534601,  61.97177379],
        [ 18.96641508,  20.53866583,  20.42740505,  20.43614804,
          22.04980155,  22.75848189,  19.886126  ,  20.07734331,
          20.17181653,  22.82409712],
        [ 19.61361955,  22.7540976 ,  20.52399636,  18.96985604,
          19.00141553,  20.34536239,  19.85689433,  19.42805363,
          19.29930741,  19.9570082 ],
        [ 19.0855977 ,  19.17656061,  18.96650625,  19.03081397,
          20.63340448,  19.67918886,  19.20896008, 

In [21]:
def nb_watch_model(model,layers):
    model.save_weights('model.cartpole')
    str_layers = ','.join([str(i) for i in layers])
    os.system('/Users/xbno/anaconda3/bin/python single_model.py -m model.cartpole -l {}'.format(str_layers))

In [22]:
def best(current_reward,num=2):
    # rank based on median then sum
    top = ((rankdata(np.median(1000/current_reward,axis=1))+rankdata(np.sum(1000/current_reward,axis=1))).max()/(rankdata(np.median(1000/current_reward,axis=1))+rankdata(np.sum(1000/current_reward,axis=1)))).argsort()[-num::][::-1]
    print('best models:',top)
    print('median:\t',np.median(current_reward,axis=1))
    print('sum:\t',np.sum(current_reward,axis=1))
    return top

def survival_of_the_fittest(current_generation,current_reward,fittest=2):
    fittest_idx = best(current_reward)
    fittest_individuals = [current_generation[idx] for idx in fittest_idx]
    fittest_weights = [[layer.get_weights()[0] for layer in individual.layers] for individual in fittest_individuals]
    return fittest_weights,fittest_individuals
    

In [24]:
def weights_to_dna(fittest_weights):
    fittest_flattened_dna = []
    fittest_weights_shapes = []
    for individual_weights in fittest_weights:
        flat = np.array([])
        weight_shapes = []
        for fit_weights in individual_weights:
            weight_shapes.append(fit_weights.shape)
            flat = np.append(flat,fit_weights)
        fittest_flattened_dna.append(flat)
        fittest_weights_shapes.append(weight_shapes)
    return list(zip(fittest_flattened_dna,fittest_weights_shapes))

In [26]:
def mutate(dna,chance=1,min_mutations=10,max_mutations=0,alter=True,max_change=.5):
    if max_mutations == 0:
        max_mutations = dna.shape[0]
    div = max_change*1000
    severity = np.random.randint(min_mutations,max_mutations)
    if np.random.rand() <= chance:
        for each_mutation in range(severity):
            base = np.random.randint(0,dna.shape[0])
            if alter:
                change = np.random.randint(0,div)/1000
                if np.random.choice(['add','sub']) == 'add':
                    dna[base] = dna[base] + change
                else:
                    dna[base] = dna[base] - change
            else:
                dna[base] = 1-(np.random.rand()*2)
    return dna

In [27]:
def reshape_dna(dna,weight_shapes):
    strand_start = 0
    reshaped_dna = []
    for shape in weight_shapes:
        ind = shape[0]*shape[1]
        reshaped_dna.append(dna[strand_start:strand_start+ind].reshape(shape))
        reshaped_dna.append(np.zeros((shape[1])))
        strand_start += ind
    return reshaped_dna

learned always remember to copy items that you're modifying in a loop because the changes will be inherited later down the line as well.

In [155]:
def create_next_generation(fittest_dna_and_shapes,num_individuals=10,layers=[4,6,5,2,1],chance=1,max_mutations=20,min_mutations=10,alter=True,max_change=.5):
    next_generation_dna_and_shapes = list(fittest_dna_and_shapes)
    
    # mutate
    for individual in range(num_individuals):
        random.shuffle(fittest_dna_and_shapes)
        dna,weight_shapes = np.copy(fittest_dna_and_shapes[0][0]),np.copy(fittest_dna_and_shapes[0][1])
        #print('dna:',fittest_dna_and_shapes[0])
        mutated_dna_and_shapes = (mutate(dna,chance=chance,min_mutations=min_mutations,max_mutations=max_mutations,alter=alter,max_change=max_change),weight_shapes)
        #print('mutated:',mutated_dna_and_shapes)
        next_generation_dna_and_shapes.append(mutated_dna_and_shapes)
    
    #print(next_generation_dna_and_shapes)
    
    # dna -> weights
    next_generation_weights = []
    for (dna,weight_shapes) in next_generation_dna_and_shapes:
        #print('next gen pairs:',(dna,weight_shapes))
        next_generation_weights.append(reshape_dna(dna,weight_shapes))
        
    # weights -> models
    next_generation = []
    for weights in next_generation_weights:
        model = create_model(layers)
        model.set_weights(weights)
        next_generation.append(model)
        
    return next_generation

tried a bunch of 'hyper' param settings and this is what I found:

1. num_best is amazing at 2, not good at 3 or 1
2. max_change from 0-1 is too much, 0-.5 or 0-.33 is better
3. the num_experiences makes a huge difference, consistency is important in choosing next best
4. also the best models are determined not only by median first important factor, but then by total points. this is key since there tend to be a lot of models at median 200 with total scores differing a lot

I'd like to modify the reward function to include having a non-moving or close to vertical pole

best seems to be 4,6,4,2

In [166]:
num_individuals = 10
num_best = 2
num_new_individuals = num_individuals - num_best
num_generations = 10
num_experiences = 10
max_change = .5
change_scale = 2.5
layers=[4,6,4,1]

print('----- first gen ------')
first_generation = create_first_gen(num_individuals=num_individuals,layers=layers)
current_generation,current_reward = generation_experience_env(env,first_generation)
fittest_weights,fittest_individuals = survival_of_the_fittest(current_generation,current_reward)
nb_watch_model(fittest_individuals[0],layers)
fittest_dna_and_shapes = weights_to_dna(fittest_weights)

try:
    for i in range(num_generations):
        print('----- next gen ------')
        # min and max mutations
        dna_length = fittest_dna_and_shapes[0][0].shape[0]
        min_mutations = int(dna_length/2)
        max_mutations = dna_length 
        
        next_generation = create_next_generation(fittest_dna_and_shapes,num_individuals=num_new_individuals,chance=1,alter=True,min_mutations=min_mutations,max_mutations=max_mutations,max_change=max_change,layers=layers)
        #next_generation = fittest_individuals+next_generation
        current_generation,current_reward = generation_experience_env(env,next_generation,trys=num_experiences)

        # scale learning rate
        total_reward = current_reward.shape[0]*current_reward.shape[1]*200
        #total_current_reward = current_reward.sum()
        max_change = ((total_reward - current_reward.sum()) / total_reward)/change_scale
        print('max change:',max_change)

        fittest_weights,fittest_individuals = survival_of_the_fittest(current_generation,current_reward)
        nb_watch_model(fittest_individuals[0],layers)
        fittest_dna_and_shapes = weights_to_dna(fittest_weights)
    
except ValueError:
    print('----- the end ------')
    print('perfect score!!!')
    print('it took {} generations of {} individuals. there were {} cartpole games played!!'.format(i,num_individuals,(i*num_individuals*num_experiences)))

----- first gen ------
best models: [2 0]
median:	 [ 10.   9. 104.   9.   9.   9.   9.  10.  10.   9.]
sum:	 [ 55.  45. 677.  46.  44.  43.  44.  48.  47.  46.]
----- next gen ------
max change: 0.23786000000000002
best models: [7 3]
median:	 [174.   10.   10.  200.   10.    9.    9.5 200.  119.5  70. ]
sum:	 [1569.  177.   97. 1946.  133.  101.   95. 1991. 1207.  791.]
----- next gen ------
max change: 0.07586000000000001
best models: [7 2]
median:	 [200.  200.  200.  180.5  10.5 123.  123.  200.  200.  191. ]
sum:	 [1998. 1945. 2000. 1719.  253. 1290. 1412. 2000. 1923. 1667.]
----- next gen ------
max change: 0.03464
best models: [9 6]
median:	 [200. 200. 200.  18. 200. 200. 200. 200. 200. 200.]
sum:	 [2000. 2000. 2000.  488. 2000. 1923. 2000. 1986. 1871. 2000.]
----- next gen ------
max change: 0.004739999999999999
best models: [8 5]
median:	 [200.  200.  200.  200.  200.  200.  200.  197.5 200.  200. ]
sum:	 [2000. 2000. 2000. 2000. 1950. 2000. 1949. 1887. 2000. 1977.]
----- next g

watch best model

In [72]:
nb_watch_model(next_generation[0],layers)

save best models

In [41]:
import pickle

In [42]:
pickle.dump(next_generation,open("good_models.p","wb"))